In [3]:
import pandas as pd

In [4]:
df = pd.read_csv('Walmart.csv')

In [5]:
# Walmart Profitability Analysis - Phase 1: Data Cleaning
# =======================================================

import pandas as pd
import numpy as np
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

# Interactive example: use the scripts.clean_and_split module from the notebook
from scripts.clean_and_split import clean_dataframe, split_dim_fact
import pandas as pd
from pathlib import Path
# adjust paths as needed
in_csv = 'Walmart.csv'
out_dir = Path('outputs/clean')
out_dir.mkdir(parents=True, exist_ok=True)
df = pd.read_csv(in_csv)
df = clean_dataframe(df)
dim_products, dim_customers, dim_stores, dim_suppliers, fact_sales = split_dim_fact(df)
df.to_csv(out_dir / 'transactions_cleaned_full.csv', index=False)
print('Wrote cleaned outputs to', out_dir)

print("="*50)

# Step 1: Load the dataset
print("\n📂 Step 1: Loading Dataset...")
df = pd.read_csv('Walmart.csv')

print(f"✅ Dataset loaded successfully!")
print(f"   • Shape: {df.shape}")
print(f"   • Columns: {len(df.columns)}")
print(f"   • Memory Usage: {df.memory_usage().sum() / 1024**2:.2f} MB")

# Step 2: Initial Data Exploration
print("\n🔍 Step 2: Initial Data Exploration...")
print(f"   • Date Range: {df['transaction_date'].min()} to {df['transaction_date'].max()}")
print(f"   • Categories: {df['category'].unique()}")
print(f"   • Store Locations: {df['store_location'].nunique()} unique locations")
print(f"   • Products: {df['product_id'].nunique()} unique products")
print(f"   • Customers: {df['customer_id'].nunique()} unique customers")

# Step 3: Check for missing values
print("\n❓ Step 3: Missing Values Analysis...")
missing_values = df.isnull().sum()
print("Missing values by column:")
for col, missing in missing_values.items():
    if missing > 0:
        pct = (missing / len(df)) * 100
        print(f"   • {col}: {missing} ({pct:.1f}%)")

if missing_values.sum() == 0:
    print("   ✅ No missing values found!")

# Step 4: Data Type Conversions
print("\n🔄 Step 4: Data Type Conversions...")

# Create a copy for processing
walmart_df = df.copy()

# Convert transaction_date to datetime
print("   • Converting transaction_date to datetime...")
walmart_df['transaction_date'] = pd.to_datetime(walmart_df['transaction_date'])
print(f"     ✅ Date range: {walmart_df['transaction_date'].min()} to {walmart_df['transaction_date'].max()}")

# Step 5: Create new calculated columns
print("\n➕ Step 5: Creating New Business Columns...")

# Total sales = quantity_sold * unit_price
walmart_df['total_sales'] = walmart_df['quantity_sold'] * walmart_df['unit_price']
print(f"   • total_sales: Min=${walmart_df['total_sales'].min():.2f}, Max=${walmart_df['total_sales'].max():.2f}")

# Calculate profit margin (assuming 25% markup)
walmart_df['profit_margin'] = walmart_df['total_sales'] * 0.25
print(f"   • profit_margin: Total=${walmart_df['profit_margin'].sum():,.2f}")

# Parse store_location into city and state
walmart_df[['city', 'state']] = walmart_df['store_location'].str.split(', ', expand=True)
print(f"   • Geographic parsing: {walmart_df['state'].nunique()} states, {walmart_df['city'].nunique()} cities")

# Extract date components for time series analysis
walmart_df['year'] = walmart_df['transaction_date'].dt.year
walmart_df['month'] = walmart_df['transaction_date'].dt.month
walmart_df['month_name'] = walmart_df['transaction_date'].dt.strftime('%B')
walmart_df['quarter'] = walmart_df['transaction_date'].dt.quarter
walmart_df['day_of_week'] = walmart_df['transaction_date'].dt.day_name()

print(f"   • Time series columns: year, month, month_name, quarter, day_of_week")

# Create forecast accuracy metric
walmart_df['forecast_accuracy'] = walmart_df['actual_demand'] / walmart_df['forecasted_demand']
print(f"   • forecast_accuracy: Mean={walmart_df['forecast_accuracy'].mean():.3f}")

# Step 6: Outlier Analysis
print("\n📊 Step 6: Outlier Analysis...")

def analyze_outliers(column):
    Q1 = walmart_df[column].quantile(0.25)
    Q3 = walmart_df[column].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    outliers = ((walmart_df[column] < lower_bound) | (walmart_df[column] > upper_bound)).sum()
    return Q1, Q3, IQR, outliers

# Analyze key numerical columns
outlier_columns = ['quantity_sold', 'unit_price', 'total_sales']
for col in outlier_columns:
    Q1, Q3, IQR, outliers = analyze_outliers(col)
    print(f"   • {col}:")
    print(f"     Q1: {Q1:.2f}, Q3: {Q3:.2f}, IQR: {IQR:.2f}")
    print(f"     Outliers: {outliers} ({outliers/len(walmart_df)*100:.1f}%)")

# Step 7: Data Quality Summary
print("\n📋 Step 7: Data Quality Summary...")
print(f"   • Total Records: {len(walmart_df):,}")
print(f"   • Total Revenue: ${walmart_df['total_sales'].sum():,.2f}")
print(f"   • Total Profit: ${walmart_df['profit_margin'].sum():,.2f}")
print(f"   • Average Transaction: ${walmart_df['total_sales'].mean():.2f}")
print(f"   • Date Coverage: {walmart_df['transaction_date'].nunique()} unique days")

# Step 8: Save cleaned dataset
print("\n💾 Step 8: Saving Cleaned Dataset...")
walmart_df.to_csv('walmart_cleaned.csv', index=False)
print(f"   ✅ Saved as 'walmart_cleaned.csv'")
print(f"   • File size: {walmart_df.memory_usage(deep=True).sum() / 1024**2:.2f} MB")
print(f"   • Columns: {len(walmart_df.columns)} ({len(walmart_df.columns) - len(df.columns)} new columns added)")

# Step 9: Preview cleaned data
print("\n👀 Step 9: Sample of Cleaned Data...")
preview_columns = ['transaction_id', 'transaction_date', 'category', 'total_sales', 
                  'profit_margin', 'city', 'state', 'month_name', 'forecast_accuracy']
print(walmart_df[preview_columns].head(10).to_string())

print("\n" + "="*50)
print("✅ PHASE 1 COMPLETE: Data Cleaning Finished!")
print(f"📊 Ready for Phase 2: SQL Database Structure")
print("="*50)

# Optional: Generate basic statistics for verification
print("\n📈 Quick Statistics Verification:")
print(f"   • Revenue by Category:")
category_stats = walmart_df.groupby('category')['total_sales'].agg(['sum', 'count', 'mean'])
for category in category_stats.index:
    revenue = category_stats.loc[category, 'sum']
    count = category_stats.loc[category, 'count']
    avg = category_stats.loc[category, 'mean']
    print(f"     {category}: ${revenue:,.2f} ({count:,} transactions, ${avg:.2f} avg)")

print(f"\n   • Revenue by State:")
state_stats = walmart_df.groupby('state')['total_sales'].sum().sort_values(ascending=False)
for state, revenue in state_stats.items():
    print(f"     {state}: ${revenue:,.2f}")

Wrote cleaned outputs to outputs\clean

📂 Step 1: Loading Dataset...
✅ Dataset loaded successfully!
   • Shape: (5000, 28)
   • Columns: 28
   • Memory Usage: 0.97 MB

🔍 Step 2: Initial Data Exploration...
   • Date Range: 1/1/2024 0:31 to 9/9/2024 8:40
   • Categories: ['Electronics' 'Appliances']
   • Store Locations: 5 unique locations
   • Products: 898 unique products
   • Customers: 3848 unique customers

❓ Step 3: Missing Values Analysis...
Missing values by column:
   • promotion_type: 3407 (68.1%)

🔄 Step 4: Data Type Conversions...
   • Converting transaction_date to datetime...
     ✅ Date range: 2024-01-01 00:31:00 to 2024-09-16 20:22:00

➕ Step 5: Creating New Business Columns...
   • total_sales: Min=$50.10, Max=$9999.25
   • profit_margin: Total=$3,815,900.36
   • Geographic parsing: 5 states, 5 cities
   • Time series columns: year, month, month_name, quarter, day_of_week
   • forecast_accuracy: Mean=1.217

📊 Step 6: Outlier Analysis...
   • quantity_sold:
     Q1: 2.00